In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

In [3]:
# Basic CSV reading
df15 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_15.csv')
df16 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_16.csv')
df17 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_17.csv')
df18 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_18.csv')
df19 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_19.csv')
df20 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_20.csv')
df21 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_21.csv')
df22 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_22.csv')
df23 = pd.read_csv(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty\missippi_poverty_23.csv')

In [4]:
df15['year'] = 2015
df16['year'] = 2016
df17['year'] = 2017
df18['year'] = 2018
df19['year'] = 2019
df20['year'] = 2020
df21['year'] = 2021
df22['year'] = 2022
df23['year'] = 2023

In [5]:
# Combine them vertically
df_all = pd.concat([df15, df16, df17, df18, df19, df20, df21, df22, df23], ignore_index=True)

In [6]:
# Split NAME column into County and State
df_all[['County', 'State']] = df_all['NAME'].str.split(',', expand=True)

C:\Users\athar\AppData\Local\Temp\ipykernel_24664\61925457.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[['County', 'State']] = df_all['NAME'].str.split(',', expand=True)
C:\Users\athar\AppData\Local\Temp\ipykernel_24664\61925457.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[['County', 'State']] = df_all['NAME'].str.split(',', expand=True)


In [7]:
# Clean up extra spaces
df_all['County'] = df_all['County'].str.strip()
df_all['State'] = df_all['State'].str.strip()

In [8]:
# Drop the original NAME column if not needed
df_all = df_all.drop(columns=['NAME'])

In [9]:
# Delete row at index 0
df_all = df_all.drop(0)

In [10]:
# Reset index after deletion
df_all = df_all.reset_index(drop=True)

In [11]:
# Remove the word "County" from County column
df_all['County'] = df_all['County'].str.replace('County', '', regex=False).str.strip()

In [12]:
# Reorder columns: GEO_ID, year, County, State, then the rest
cols = df_all.columns.tolist()

In [13]:
# Ensure order
new_order = ['GEO_ID', 'year', 'County', 'State'] + [col for col in cols if col not in ['GEO_ID', 'year', 'County', 'State']]


In [14]:

df_all = df_all[new_order]

In [15]:
keep = [
    "GEO_ID", "NAME", "State", "County", "year",
    "S1701_C01_001E",      # Total Population
    "S1701_C02_001E",       # Population Below Poverty
    "S1701_C03_001E",       #Percent Below Poverty
    "S1701_C01_024E",       #Adults with High School
    "S1701_C03_024E",       #Percent Adults with High School
    "S1701_C01_026E",       #Adults with College Degree
    "S1701_C03_026E"        #Percent Adults with College Degree
]

In [16]:
# Filter to the columns that exist, then rename
existing = [c for c in keep if c in df_all.columns]

In [17]:
renamed = {
    "S1701_C01_001E": "Total Population",
    "S1701_C02_001E": "Population Below Poverty",     
    "S1701_C03_001E": "Percent Below Poverty",      
    "S1701_C01_024E": "Adults with High School",       
    "S1701_C03_024E": "Percent Adults with High School",       
    "S1701_C01_026E": "Adults with College Degree",       
    "S1701_C03_026E": "Percent Adults with College Degree"       
}

In [18]:
df_out = df_all[existing].rename(columns=renamed)

In [19]:
df_out.head()

,GEO_ID,State,County,year,Total Population,Population Below Poverty,Percent Below Poverty,Adults with High School,Percent Adults with High School,Adults with College Degree,Percent Adults with College Degree
0,0500000US28001,Mississippi,Adams,2015,29586,8898,30.1,7092,24.6,3850,7.7
1,0500000US28003,Mississippi,Alcorn,2015,36608,7752,21.2,9044,22.6,4049,5.8
2,0500000US28005,Mississippi,Amite,2015,12727,2852,22.4,3348,18.0,1118,9.2
3,0500000US28007,Mississippi,Attala,2015,18871,4908,26.0,3889,21.7,1874,6.4
4,0500000US28009,Mississippi,Benton,2015,8389,1764,21.0,2154,11.3,599,1.2


In [20]:
keep_county = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\appalachian_counties.csv")

In [21]:
keep_mississippi = keep_county[keep_county["state"] == "Mississippi"]

In [22]:
mississippi_counties = set(keep_mississippi["county"])

In [23]:
# Keep only rows in df_out where 'County' is in that list
df_out5 = df_out[df_out["County"].isin(mississippi_counties)]

In [24]:
df_out5.head()

,GEO_ID,State,County,year,Total Population,Population Below Poverty,Percent Below Poverty,Adults with High School,Percent Adults with High School,Adults with College Degree,Percent Adults with College Degree
1,0500000US28003,Mississippi,Alcorn,2015,36608,7752,21.2,9044,22.6,4049,5.8
4,0500000US28009,Mississippi,Benton,2015,8389,1764,21.0,2154,11.3,599,1.2
6,0500000US28013,Mississippi,Calhoun,2015,14501,3763,25.9,3346,16.7,1109,4.7
8,0500000US28017,Mississippi,Chickasaw,2015,17074,4493,26.3,4126,21.0,1210,6.1
9,0500000US28019,Mississippi,Choctaw,2015,8198,2147,26.2,1870,18.0,758,6.9


In [25]:
df_out5.shape


(216, 11)

In [26]:
df_out5.to_csv("df_out5.csv", index=False)
